# Script permettant de générer la carte exonique (exon map) des gènes d'intérêt
Input: le json file génnéré à partir des réponse API Ensemble Lookup. Plusieurs réponse peuvent être dans un seul json, une par ligne, le json est mergé.  
Output: exon_map.tab fichier texte tabulation contenant la position de chaque exon et la séquence pour chaque transcript d'intérêt

In [1]:
# Importer les données: json + correspondance uniprot/refseq + genomic
import json
import pandas as pd

# Pour merger les json entre eux
def merge_two_dicts(x, y):
    z = x.copy()
    z.update(y)    
    return z

# Pour importer les json
def importJson(filePath):
    data_dict = []
    with open(filePath) as json_data:
        for i in json_data:
            data_dict.append(json.loads(i))
    MyJsonFull = data_dict[0]
    for i in data_dict[1:]:
        MyJsonFull = merge_two_dicts(MyJsonFull, i)
    return MyJsonFull

# Renvoie une string contenant la ligne à écrire dans le fichier (position exon, séquence...)
def catExonPos(MyJsonFull, seqID, dict_Genomic):
    exonString = []
    if MyJsonFull[seqID]["strand"] == -1:
        ABSOLUTE_POST = MyJsonFull[seqID]["end"]
        exon_list = MyJsonFull[seqID]["Exon"]
        counter = 1
        for i in exon_list:
            stop = str(abs(i['end']-ABSOLUTE_POST))
            start = str(abs(i['start']-ABSOLUTE_POST)+1)
            res = [val for key, val in my_Genomic.items() if seqID in key]
            exonString.append(seqID+ "\tExon\t"+str(counter)+"\t"+stop+"\t"+start+"\t"+res[0][int(stop):int(start)])
            counter+=1

    if MyJsonFull[seqID]["strand"] == 1:
        ABSOLUTE_POST = MyJsonFull[seqID]["start"]
        exon_list = MyJsonFull[seqID]["Exon"]
        counter = 1
        for i in exon_list:
            start = str(abs(i['start']-ABSOLUTE_POST))
            stop = str(abs(i['end']-ABSOLUTE_POST)+1)
            res = [val for key, val in dict_Genomic.items() if seqID in key]
            exonString.append(seqID+ "\tExon\t"+str(counter)+"\t"+start+"\t"+stop+"\t"+res[0][int(start):int(stop)])
            counter+=1
    return(exonString)

# Importe un fichier fasta en dictionnaire titre:séquence
def fasta2List(pathFasta):
    f = open(pathFasta, "r")
    title = []
    seq = []
    seq_temp = []
    for line in f:
        if line[0] == ">":
            seq.append(''.join(seq_temp).replace("\n", ""))
            title.append(line.replace("\n", ""))
            seq_temp = []
        else:
            seq_temp.append(line)
    seq.append(''.join(seq_temp).replace("\n", ""))
    seq.pop(0)
    dictionary = dict(zip(title, seq))
    return dictionary

In [3]:
ID_file = pd.read_csv("../../data/mismatch-analysis/uniprot-exon-map/transcript_ensembl.tab", sep = "\t")
jsonFile = importJson('../../data/mismatch-analysis/lookup_newline.json')
my_Genomic = fasta2List("../../data/mismatch-analysis/genomics_new.fa")

FileNotFoundError: [Errno 2] No such file or directory: '../../data/mismatch-analysis/lookup_newline.json'

In [ ]:
with open("../../data/mismatch-analysis/uniprot-exon-map/Exon_map.tab", "w") as exon_file:
    for i in ID_file.iloc[:,1]:
        uniprot_name = ID_file.loc[ID_file["To"] == i].iloc[0,0]
        for exon in catExonPos(jsonFile, i, my_Genomic):
            exon_file.write(uniprot_name+"\t"+exon+"\n")

In [ ]:
# On fait maintenant l'intron map pour trouver les petits introns
Exon_Map = pd.read_csv("../../data/mismatch-analysis/uniprot-exon-map/Exon_map.tab", sep="\t", header=None)
protein_list = list(set(Exon_Map.iloc[:,0].to_list()))

Intron_Map_light = open("../../data/mismatch-analysis/uniprot-exon-map/Intron_map_light.tab", "w")

for prot in protein_list:
    subset = Exon_Map.loc[Exon_Map[0]==prot]
    intron_number = len(subset.index)-1
    for i in range(intron_number):
        intron_start = str(int(subset.iloc[i,5]))
        intron_stop = str(int(subset.iloc[i+1,4]))
        ensembl_ID = str(subset.iloc[i,1])
        Intron_Map_light.write(prot+"\t"+ensembl_ID+"\tIntron\t"+str(i+1)+"\t"+intron_start+"\t"+intron_stop+"\n")

Intron_Map_light.close()